In [5]:
import sys
sys.path.append('../')

import tquant as tq
import tensorflow as tf

from datastore import *
import datetime

tq.Settings.evaluation_date = date(2024, 4, 30)

In [3]:
# rc0 = tq.RateCurve(ir_eur_curve_estr['daycount'].values/365, ir_eur_curve_estr['quote'].values)
rc = tq.RateCurveTest(ir_eur_curve_estr['daycount'].values/365, ir_eur_curve_estr['quote'].values, 'LINEAR')

In [8]:
## class helper
## input class helper
currency = tq.Currency.EUR
start_delay = 2
fixing_days = 2
period_fix_units = 1
period_fix = tq.TimeUnit.Years
period_flt_units = 6
period_flt = tq.TimeUnit.Months
bd_convention = tq.BusinessDayConvention.ModifiedFollowing
notional = 1.0
day_count_convention_fix = tq.DayCounterConvention.Actual360
day_count_convention_flt = tq.DayCounterConvention.Actual360

#### function build helper
#### input function build  
trade_date = tq.Settings.evaluation_date
quote = 0.01
term = 2
term_units = tq.TimeUnit.Years

if currency == tq.Currency.EUR:
    calendar = tq.TARGET()
else:
    raise TypeError("Calendar is not present for currency")

start_date = calendar.advance(trade_date,
                              start_delay, 
                              tq.TimeUnit.Days, 
                              bd_convention)
maturity = calendar.advance(start_date, term, term_units, bd_convention)

schedule_generator = tq.ScheduleGeneratorTest(calendar, bd_convention)
schedule_fix = schedule_generator.generate(start_date, 
                                           maturity, 
                                           period_fix_units, period_fix)
schedule_float = schedule_generator.generate(start_date,
                                             maturity,
                                             period_flt_units,
                                             period_flt)

start_fix = []
end_fix = []
pay_fix = []
for i in range(len(schedule_fix) - 1):
    start_fix.append(schedule_fix[i])
    end_fix.append(schedule_fix[i + 1])
    pay_fix.append(calendar.adjust(schedule_fix[i + 1], bd_convention))

day_count_fix = tq.DayCounter(day_count_convention_fix)
day_count_flt = tq.DayCounter(day_count_convention_flt)

start_flt = []
end_flt = []
pay_flt = []
for i in range(len(schedule_float) - 1):
    start_flt.append(schedule_float[i])
    end_flt.append(schedule_float[i + 1])
    pay_flt.append(calendar.adjust(schedule_float[i + 1], bd_convention))

fixing_dates = []
fixing_rates = []


index = tq.OvernightIndex('ESTR', calendar)

ois_test = tq.OisTest(schedule_float, 
                      schedule_fix,
                      notional,
                      index,
                      quote,
                      day_count_fix,
                      day_count_flt,
                      tq.Currency.EUR)

In [9]:
eng_ois_test = tq.SwapAnalyticEngine(ois_test)
eng_ois_test.price(rc, rc, trade_date)

<tf.Tensor: shape=(), dtype=float64, numpy=-0.045277554423039795>